In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
from pandas import DataFrame
import numpy as np

In [ ]:
train = pd.read_csv("../input/instacart-market-basket-analysis/order_products__train.csv")
prior = pd.read_csv("../input/instacart-market-basket-analysis/order_products__prior.csv")
orders = pd.read_csv("../input/instacart-market-basket-analysis/orders.csv")
products = pd.read_csv("../input/instacart-market-basket-analysis/products.csv")
aisles = pd.read_csv("../input/instacart-market-basket-analysis/aisles.csv")
departments = pd.read_csv("../input/instacart-market-basket-analysis/departments.csv")

In [ ]:
prior = pd.concat([train, prior])

In [ ]:
#prior[:10]

prior1 => prior과 train을 결합하고, products를 결합함. order_id / product_id / add_to_cart_order / reordered / product_name / aisle_id / department_id 로 구성되어 있음.**

In [ ]:
prior1 = prior.join(products.set_index('product_id'), on='product_id')

In [ ]:
from collections import Counter as cc

pro_name = cc(prior1['product_name'])
A = pro_name.most_common()[:1000]
#print(A)

In [ ]:
A = DataFrame(A)
A.columns = ["product_name","product_count"]

In [ ]:
#print(A)

In [ ]:
top10 = A.join(prior1.set_index('product_name'), on='product_name')

In [ ]:
len(top10)

In [ ]:
All_top10 = pd.merge(top10, orders, how='left')

In [ ]:
#All_top10

All_top10에서 user_order을 만들고 user_id와 order_number, eval_set, product_count, product_id 를 삭제 

In [ ]:
All_top10['user_order'] =All_top10['user_id'] + All_top10['order_number'] * 0.1

In [ ]:
All_top10.drop(['eval_set', 'product_count', 'product_id', 'user_id', 'order_number'], axis='columns', inplace=True)
All_top10.head()

product_name별로 그룹지음

In [ ]:
group_top10 = All_top10["product_name"].groupby(All_top10['user_order'])
group_top10 = pd.DataFrame(group_top10)

In [ ]:
group_top10.columns = ['user_order','product_list']

In [ ]:
group_top10.head()

In [ ]:
product = [list(s) for s in group_top10.product_list]

In [ ]:
#product

In [ ]:
len(product)

In [ ]:
len(group_top10)

Word2Vec 시작

In [ ]:
from gensim.models.word2vec import Word2Vec

In [ ]:
%%time
model = Word2Vec(product)

In [ ]:
model.init_sims(replace=True)

similarity : 두 단어의 유사도 계산
most_similar : 가장 유사한 단어를 출력

In [ ]:
model.wv.similarity('Banana', 'Organic Whole Milk')

In [ ]:
model.wv.similarity('Red Mango', 'Honeydew Melon')

In [ ]:
model.wv.similarity('Bag of Organic Bananas', 'Organic Whole Milk')

In [ ]:
model.wv.most_similar("Banana")

In [ ]:
model.wv.most_similar("Organic Avocado")

In [ ]:
model.wv.most_similar("Organic Hass Avocado")

In [ ]:
model.wv.most_similar("Bag of Organic Bananas")

In [ ]:
model.wv.most_similar("Organic Gala Apples")

In [ ]:
model.wv.most_similar("Apple Honeycrisp Organic")

In [ ]:
model.wv.most_similar("Organic Strawberries")

In [ ]:
model.wv.most_similar("Seedless Red Grapes")

In [ ]:
model.wv.most_similar("Granny Smith Apples")

In [ ]:
model.wv.most_similar("Fat Free Milk")

In [ ]:
model.wv.most_similar("Original Orange Juice")

In [ ]:
model.wv.most_similar("Honey Nut Cheerios")

In [ ]:
model.wv.most_similar("Lemonade")

In [ ]:
model.wv.most_similar("Organic Blueberries")

In [ ]:
model.wv.most_similar("Blueberries")

In [ ]:
model.wv.most_similar("Granny Smith Apples")

In [ ]:
model.wv.most_similar("Organic Half & Half")

In [ ]:
model.wv.most_similar("Half & Half")

In [ ]:
model.wv.most_similar("Organic Fuji Apple")

In [ ]:
model.wv.most_similar("Honeycrisp Apple")

In [ ]:
model.wv.most_similar("Strawberries")

In [ ]:
model.wv.most_similar("Organic Baby Spinach")

In [ ]:
model.wv.most_similar("Limes")

In [ ]:
model.wv.most_similar("Extra Virgin Olive Oil")

In [ ]:
model.wv.most_similar("Asparagus")

In [ ]:
model.wv.most_similar("Organic Whole Milk")

In [ ]:
model.wv.most_similar("Organic Reduced Fat Milk")

In [ ]:
model.wv.most_similar("Whipped Cream Cheese")

In [ ]:
model.wv.most_similar("Organic Baby Spinach")

# 시각화

  low_dim_embs 내부 갯수와 n_components가 같아야 한다.
  
  n_components : 차원. default는 2.
    
  perplexity : 가장 가까운 이웃 갯수. 보통 5~50. default는 30.
     
  n_iter : 최적화에 사용할 반복 횟수. 최소 200. default는 1000.
     
  init : embedding 초기화 방법. random과 pca 중에서 선택. pca가 보다 안정적. default는 random.



In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

def tsne_plot(model):
    labels = []
    tokens = []

    for word in model.wv.vocab:
        tokens.append(model[word])
        labels.append(word)
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(128, 128)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()

In [ ]:
#tsne_plot(model)

In [ ]:
model.save('product_model_top1000.model')